In [3]:
%cd C:\Users\Mo Ragheb\Downloads\-Python-3-Bootcamp-master\Labs\Scrapper

from pinnacle.apiclient import  APIClient
from pinnacle.exceptions import  PinnacleError
from pinnacle.resources.baseresource import BaseResource
from pinnacle.resources.baseresource import BaseResource
from dateutil import parser
from datetime import  datetime,timedelta
import time
from ipywidgets import interact,interactive,fixed,widgets
from IPython.display import display,clear_output
from simplejson.decoder import JSONDecodeError
import logging
from pinnacle.resources.referencedataresources import (
    SportsDetails,
    CurrencyDetails,
    PeriodDetails,
    LeagueDetails,
)

from pinnacle.resources.accountresources import AccountDetails

from pinnacle.resources.marketdataresources import (
    FixtureDetails,
    SettledFixtureDetails,
    SpecialFixtureDetails,
    SettledSpecialFixtureDetails,
    OddsDetails,
    SpecialOddsDetails,
    LineDetails,
    SpecialLineDetails,
    InRunningDetails,
)

from pinnacle.resources.bettingresources import (
    BetDetails,
    PlaceBetDetails,
    PlaceSpecialBetDetails,
)

import sys
from openpyxl import load_workbook
from openpyxl import Workbook
from openpyxl.styles import PatternFill
import pinnacle.enums as enums
from IPython.core.debugger import set_trace
from collections import  OrderedDict
import threading
from queue import Queue
from Classes.Comp import Comp
from HelperMethods import Methods
import os.path
import re
import numpy as np
from fuzzywuzzy import fuzz


C:\Users\Mo Ragheb\Downloads\-Python-3-Bootcamp-master\Labs\Scrapper


In [ ]:
api = APIClient('MR1009178', 'Keymuse_13')

pinnyconfigfile=r"C:\Users\Mo Ragheb\Downloads\Data minings\Stats\PinnacleLeagues.xlsx"
pinnyleagues= OrderedDict()
pinnysports=OrderedDict()
Logfilelocation="C:\\Users\Mo Ragheb\\Downloads\Data minings\\Scrapper logs\\pinny1"+str (time.strftime("%d-%m-%Y"+"test.txt"))
#Logfilelocation="test.txt"
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)
logging.basicConfig(level = logging.DEBUG, format = '%(asctime)s  %(levelname)-10s %(processName)s %(funcName)s   %(message)s', filename = Logfilelocation,filemode="w" )
PinnyUserNAme="MR1009178" 
PinnyPass="Keymuse_13"
PinnyEvents=[]
tempevent=None
Eventsqueue=Queue()
Comp.buildallleagues()
DataMiningspath="C:\\Users\\Mo Ragheb\\Downloads\Data minings\\"

if "availableBalance" in api.account.get_account().keys():
    logging.info("Connection to pinnacle is a sucess")
else:
        logging.info("Pinnace Invlaid Credntials")


In [ ]:
class pinnyEvent:
    global pinnyleagues
    def __init__(self,rotnumber=0,sportid=0,leagueid=0,hometeam="",awayteam="",startdate=datetime.now().date(),eventid=0,homeodds=0,awayodds=0,drawodds=0,totalsover=0,totalsunder=0,httotlasover=0,httunder=0,attover=0,attunder=0,homespread=0,awayspread=0,spread=0,totals=0,hteamtotals=0,ateamtotls=0,homepitcher="",awaypitcher=""):
        self.leagueid=leagueid
        self.hometeam=hometeam
        self.awayteam=awayteam
        self.startdate=startdate
        self.rotnumber=rotnumber
        self.eventid=eventid
        self.spread=spread
        self.totals=totals
        self.homeodds=homeodds
        self.awayodds=awayodds
        self.totalsover=totalsover
        self.totalsunder=totalsunder
        self.homespread=homespread
        self.awayspread=awayspread
        self.httotlasover=httotlasover
        self.httunder=httunder
        self.attover=attover
        self.attunder=attunder
        self.hteamtotals=hteamtotals
        self.ateamtotls=ateamtotls
        self.homepitcher=homepitcher
        self.awaypitcher=awaypitcher
        self.sportid=sportid
    def __str__(self):
        return("League : " + str(pinnyleagues[self.leagueid][0])+" Event ID:"+str(self.eventid)+ " Rotation number :" +self.rotnumber+"  Home team : "+self.hometeam+" Away team : "+self.awayteam +" startdate:" +str(self.startdate) )
    

In [6]:
class threadsafe_iter:
    """Takes an iterator/generator and makes it thread-safe by
    serializing call to the `next` method of given iterator/generator.
    """
    def __init__(self, it):
        self.it = it
        self.lock = threading.Lock()

    def __iter__(self):
        return self

    def __next__(self):
        with self.lock:
                return self.it.__next__()
    def __str__():
        return str(self)



In [7]:
api.account.get_account()

{'Latency': 0.105719,
 'TIMESTAMP': '2019-08-14 10:50:55.029469',
 'code': 'NO_API_ACCESS',
 'message': 'Account not permitted to access the API'}

In [150]:
def threadsafe_generator(f):
    """A decorator that takes a generator function and makes it thread-safe.
    """
    def g(*a, **kw):
        return threadsafe_iter(f(*a, **kw))
    return g

In [151]:

def loop(func, n):
    """Runs the given function n times in a loop.
    """
    for i in range(n):
         func()

In [152]:
def runthreads(f, repeats=1000, nthreads=10):
    """Starts multiple threads to execute the given function multiple
    times in each thread.
    """
    # create threads
    threads = [threading.Thread(target=loop, args=(f, repeats))
               for i in range(nthreads)]

    # start threads
    for t in threads:
        t.start()

    # wait for threads to finish
    for t in threads:
        t.join()

In [153]:
def buildpinnysports():
    for sport in api.reference_data.get_sports():
        pinnysports[sport["id"]]=sport["name"]

In [ ]:
def buildpinnyleaguesdict():
    wb=load_workbook(filename=pinnyconfigfile)
    sheet   =wb.active
    pinnyleagues.clear()
    for row in sheet.iter_rows(row_offset=1):
            if(row[0].value!=None):  
                pinnyleagues[int(row[0].value)]=[row[1].value,row[2].value]
            else:
                break
    
        

In [ ]:
buildpinnyleaguesdict()

In [ ]:
def exportallsportleaguesfrompinny(sportid):
    leagues=(api.reference_data.get_leagues(sport_id=sportid))
    wb=load_workbook(filename = fname)
    if len(pinnysports)==0:
        buildpinnysports()
    if  "my "+ pinnysports[sportid] +" Leagues"  not in wb.get_sheet_names():
        sheet=wb.create_sheet("my "+ pinnysports[sportid] +" Leagues")
    else:
        sheet=wb.get_sheet_by_name("my "+ pinnysports[sportid] +" Leagues")
    #sheet.title=
    sheet.cell(row=1, column=1).value ="League ID"
    sheet.cell(row=1, column=2).value ="League Name"

    #sheet.cell(row=1, column=3).value ="VS"
    for key, league in  Oddsle:
           sheet.cell(row=index+2, column=1).value =key
           sheet.cell(row=index+2, column=2).value =le
    wb.save(fname)
     
        

In [178]:
def getthepinnyoddsforallsports():
    #set_trace()
    Eventsqueue.empty()
    if len(pinnyleaguese)==0:
        buildpinnyleaguesdict()
    if len(pinnysports)==0:
        buildpinnysports()

   # threads = [threading.Thread(target=Fixturesgen.next(), args=(, )) for i in range(numofthreads)]
    for  key in pinnyleagues.keys():
       gen=getthepinnyodds(key)
       try :
           
            runthreads(gen.__next__,len(list(gen)),4)
       except StopIteration:
            continue


In [141]:
@threadsafe_generator
def getthepinnyodds(leagueid):
    
        try:
            sportid=pinnyleagues[leagueid][1]
            fixtures=api.market_data.get_fixtures(sport_id=sportid,is_live=False,league_ids=[leagueid],since=1000)
            for event  in fixtures["league"][0]["events"]:
                tempevent=pinnyEvent()
                tempevent.sportid=sportid
                #print(event["starts"])
                tempevent.leagueid=fixtures["league"][0]["id"]

                tempevent.startdate=parser.parse(event["starts"]).date()

                tempevent.hometeam=event["home"].lower()
                tempevent.awayteam=event["away"].lower()
                tempevent.rotnumber=event["rotNum"]
                tempevent.eventid=event["id"]
                #getdate only for next 7 days
                if tempevent.startdate> datetime.now().date() +  timedelta(days=7):
                    logging.info("event is ,ore than 7 days ahead "+str(tempevent))
                    continue

                #Money Line
            #     homeline=api.market_data.get_line(bet_type=enums.BetType.MoneyLine.value,odds_format=enums.OddsFormat.Decimal.value,event_id=tempevent.eventid,league_id=tempevent.leagueid,sport_id=29,team=enums.TeamType.Team1.value,period_number=0)
            #     awayline=api.market_data.get_line(bet_type=enums.BetType.MoneyLine.value,odds_format=enums.OddsFormat.Decimal.value,event_id=tempevent.eventid,league_id=tempevent.leagueid,sport_id=29,team=enums.TeamType.Team2.value,period_number=0)
            #     drawline=api.market_data.get_line(bet_type=enums.BetType.MoneyLine.value,odds_format=enums.OddsFormat.Decimal.value,event_id=tempevent.eventid,league_id=tempevent.leagueid,sport_id=29,team=enums.TeamType.Draw.value,period_number=0)
            #     print(event.keys())
            #     # Totals
            #     totaloverline=api.market_data.get_line(bet_type=enums.BetType.Total.value,odds_format=enums.OddsFormat.Decimal.value,event_id=tempevent.eventid,league_id=tempevent.leagueid,sport_id=29,side=enums.Side.Over.value,period_number=0)
            #     totalunderline=api.market_data.get_line(bet_type=enums.BetType.Total.value,odds_format=enums.OddsFormat.Decimal.value,event_id=tempevent.eventid,league_id=tempevent.leagueid,sport_id=29,side=enums.Side.Under.value,period_number=0)   
            #     # Spread
            #     HomeAHline=api.market_data.get_line(bet_type=enums.BetType.Spread.value,odds_format=enums.OddsFormat.Decimal.value,event_id=tempevent.eventid,league_id=tempevent.leagueid,sport_id=29,team=enums.TeamType.Team1.value,period_number=0)
            #     awayAHLine=api.market_data.get_line(bet_type=enums.BetType.Spread.value,odds_format=enums.OddsFormat.Decimal.value,event_id=tempevent.eventid,league_id=tempevent.leagueid,sport_id=29,team=enums.TeamType.Team2.value,period_number=0)

            #     #TeamTotals
            #     teamtotaloverline=api.market_data.get_line(bet_type=enums.BetType.TeamTotal.value,odds_format=enums.OddsFormat.Decimal.value,event_id=tempevent.eventid,league_id=tempevent.leagueid,sport_id=29,side=enums.Side.Over.value,team=enums.TeamType.Team1.value,period_number=0)
            #     teamtotalunderine=api.market_data.get_line(bet_type=enums.BetType.TeamTotal.value,odds_format=enums.OddsFormat.Decimal.value,event_id=tempevent.eventid,league_id=tempevent.leagueid,sport_id=29,side=enums.Side.Under.value,team=enums.TeamType.Team1.value,period_number=0)    
            #     teamtotaloverline=api.market_data.get_line(bet_type=enums.BetType.TeamTotal.value,odds_format=enums.OddsFormat.Decimal.value,event_id=tempevent.eventid,league_id=tempevent.leagueid,sport_id=29,side=enums.Side.Over.value,team=enums.TeamType.Team2.value,period_number=0)
            #     teamtotalunderine=api.market_data.get_line(bet_type=enums.BetType.TeamTotal.value,odds_format=enums.OddsFormat.Decimal.value,event_id=tempevent.eventid,league_id=tempevent.leagueid,sport_id=29,side=enums.Side.Under.value,team=enums.TeamType.Team2.value,period_number=0)    

                try :
                   odds=api.market_data.get_odds(event_ids=[tempevent.eventid],odds_format=enums.OddsFormat.Decimal.value,since=10000,sport_id=sportid,league_ids=[tempevent.leagueid])
                   #baseball pitchers
                   if sportid == 3:
                    tempevent.homepitcher=event["homePitcher"]
                    tempevent.awaypitcher=event["awayPitcher"]
                   #spread
                   try:
                       tempevent.spread=odds["leagues"][0]["events"][0]["periods"][0]["spreads"][0]["hdp"]
                       tempevent.homespread=odds["leagues"][0]["events"][0]["periods"][0]["spreads"][0]["home"]   
                       tempevent.awayspread=odds["leagues"][0]["events"][0]["periods"][0]["spreads"][0]["away"]                     
                   except  KeyError:
                       logging.info("no spread for  this event"+str(tempevent))
                       
                   #total
                   try:
                       tempevent.totals=odds["leagues"][0]["events"][0]["periods"][0]["totals"][0]["points"]
                       tempevent.totalsover=odds["leagues"][0]["events"][0]["periods"][0]["totals"][0]["over"]
                       tempevent.totalsunder=odds["leagues"][0]["events"][0]["periods"][0]["totals"][0]["under"] 
                   except KeyError:
                       logging.info("no totals for  this event"+str(tempevent))
                     
                
                   #ML odds
                   tempevent.homeodds=odds["leagues"][0]["events"][0]["periods"][0]["moneyline"]["home"]
                   tempevent.awayodds=odds["leagues"][0]["events"][0]["periods"][0]["moneyline"]["away"]
                   if sportid==29:                            #soccer  has draw 
                       tempevent.drawodds=odds["leagues"][0]["events"][0]["periods"][0]["moneyline"]["draw"]
                   #total odds
                   try :
                       tempevent.hteamtotals=odds["leagues"][0]["events"][0]["periods"][0]["teamTotal"]["home"]["points"]
                       tempevent.ateamtotls=odds["leagues"][0]["events"][0]["periods"][0]["teamTotal"]["away"]["points"]
                       tempevent.httotlasover=odds["leagues"][0]["events"][0]["periods"][0]["teamTotal"]["home"]["over"]
                       tempevent.httunder=odds["leagues"][0]["events"][0]["periods"][0]["teamTotal"]["home"]["under"]                    
                       tempevent.attover=odds["leagues"][0]["events"][0]["periods"][0]["teamTotal"]["away"]["over"]   
                       tempevent.attunder=odds["leagues"][0]["events"][0]["periods"][0]["teamTotal"]["away"]["under"]  
                   
                   except KeyError as e:            # Team Totals are not yet up
                        logging.info("no team team total for  this event"+str(tempevent))
                   logging.info("odds exracted for this event"+str(tempevent))  
                  
                except Exception as e:
                    logging.debug("Odds not found for this game"+" "+str(tempevent) +str(e))
                    continue
                try:
                  # set_trace()
                   #print(tempevent)
                   Eventsqueue.put(tempevent)
                   yield(tempevent) 
                   logging.info("odds yielded for this event"+str(tempevent))                               
                except StopIteration:
                    break
        except JSONDecodeError  as e:
                logging.debug("no fixture found for this league "+ pinnyleagues[leagueid][0])




                #if(line["status"]=="SUCCESS"):
                    #print(line["price"])




In [142]:
def buttonclick(change):
    button.disabled=True
    t.disabled=True
    gen=getthepinnyodds(leagueid=int(leagueidlabel.value))
    while True:
        try:
            if len(pinnysports) ==0:
                buildpinnysports()
            if len(pinnyleagues) ==0::
                buildpinnyleaguesdict()    
                
            print(gen.__next__())
        except StopIteration:
            break
    button.disabled=False
    t.disabled=False
def changelabelname(change):
    leagueidlabel.value=str(list(pinnyleagues.keys())[t.index])
t=widgets.Select(options=pinnyleagues.values())
t.layout.width="initial"
button=widgets.Button()
button.description="Update the odds"
button.style.button_color="skyblue"
button.on_click(buttonclick)
leagueidlabel=widgets.Label("LeagueID")
#leagueidlabel.values="gggg"
t.observe(changelabelname,names=["label"])





In [143]:
clear_output()
display(t,button,leagueidlabel)


A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

In [144]:
getthepinnyoddsforallsports()

KeyError: 'league'

In [137]:
tempqueue=Eventsqueue.queue.copy()

In [138]:
def updatetheextractedodds():
    while len(tempqueue)>0:
        tempevent=tempqueue.pop()
        #print(tempevent)
        #mma
        if tempevent.leagueid in [1624,1619]:
            updateCSVwithodds(Comp(sport="mma"),tempevent)
        #boxing 
        else:
             updateCSVwithodds(Comp.leagues[tempevent.leagueid],tempevent)



In [139]:
def updateCSVwithodds(comp=None,pinnyevent=None):

    if comp is None : 
        logging.info("no file for this extracted odds")
        return "no file for this extracted odds"
    compname= comp.competitionname 
    if compname=="wnba" or compname!="2-bundesliga" :
        compname= Methods.replacecompetionanme(compname)   
    if comp.country=="egypt":
        fname="egyptian-"+compname.lower().replace(" ","-").replace(":","-") +" "+ comp.season+".xlsx"
    elif comp.country=="russia":
        fname="russian-"+compname.lower().replace(" ","-").replace(":","-") +" "+ comp.season+".xlsx"        
    elif comp.sport=="handball":
        fname="handball-"+compname.lower().replace(" ","-").replace(":","-") +" "+ comp.season+".xlsx"
    elif comp.sport=="mma" or comp.sport=="boxing" :
        fname=  comp.sport + " " + comp.season+".xlsx"
    elif compname=="super-cup":
        fname= comp.country + " " + compname.lower().replace(" ","-").replace(":","-") + " "+comp.season+".xlsx" 
    elif compname=="premiership" and comp.sport=="rugby-union":
        fname="aviva "+compname.lower().replace(" ","-").replace(":","-") +" "+ comp.season+".xlsx"       
    else:
        fname=compname.lower().replace(" ","-").replace(":","-") +" "+ comp.season+".xlsx"

    logging.info("File name is :{0}".format(fname)) 

    if not os.path.exists(DataMiningspath+fname):
        logging.info("{0} {1} {2}fixtures not saved yet".format(comp.sport,comp.country,compname))
        return comp.sport+" "+ comp.country+" "+ compname +" fixtures not saved yet"
    wb = load_workbook(filename = DataMiningspath+fname)
    sheet=wb.active
    sheet.column_dimensions['L'].hidden= True
    sheet.column_dimensions['M'].hidden= True
    counter=0
    logging.info("file openned")
    eventdate=pinnyevent.startdate
    margin = timedelta(days=1)
    for index,row in enumerate(sheet.iter_rows()):
                #set_trace()
                if  index!=0 and (eventdate-margin<=parser.parse(row[4].value,dayfirst=True).date()<=eventdate+margin):
                     player1text= row[1].value.lower().split("(")[0].replace(".","") if "(" in row[1].value else row[1].value.lower().replace(".","")
                     player2text=row[2].value.lower().split("(")[0].replace(".","") if "(" in row[2].value else row[2].value.lower().replace(".","")
                     if(fuzz.token_sort_ratio(player1text,tempevent.hometeam)>=90):
                            print(player1text)
                    
            
    sheet=Methods.adjustwidth(sheet)
    try:  
        wb.save(DataMiningspath+fname)
        logging.info("league {0} : {1} {2} matches updated".format(comp.country,compname,str(str(counter))))
        return  ("league " + comp.country +" :"+compname +" "+ str(counter)+ " matches updated")
    except Exception as e:
        return(fname + "is open by other person")

In [140]:

updatetheextractedodds()

AttributeError: 'NoneType' object has no attribute 'hometeam'

In [ ]:

Comp.leagues[2406].competitionname

In [179]:
for comp in Comp.leagues.values():
    print (comp)

Competiton is premier-league the sport is soccer
Competiton is championship the sport is soccer
Competiton is efl-cup the sport is soccer
Competiton is fa-cup the sport is soccer
Competiton is fa-community-shield the sport is soccer
Competiton is ligue-1 the sport is soccer
Competiton is ligue-2 the sport is soccer
Competiton is coupe-de-la-ligue the sport is soccer
Competiton is coupe-de-france the sport is soccer
Competiton is super-cup the sport is soccer
Competiton is bundesliga the sport is soccer
Competiton is 2-bundesliga the sport is soccer
Competiton is dfb-pokal the sport is soccer
Competiton is super-cup the sport is soccer
Competiton is serie-a the sport is soccer
Competiton is coppa-italia the sport is soccer
Competiton is super-cup the sport is soccer
Competiton is laliga the sport is soccer
Competiton is copa-del-rey the sport is soccer
Competiton is super-cup the sport is soccer
Competiton is caf-champions-league the sport is soccer
Competiton is caf-confederations-cup 

In [ ]:
[l["name"]+str(l["id"] )for l in api.reference_data.get_leagues(sport_id=29)] 

In [ ]:
fixtures=getthepinnyodds(leagueid=1977)
    

In [ ]:
next(fixtures)

In [133]:
st="liz carmouche"
str2="Carmouche L. (Usa)"


In [134]:
fuzz.token_sort_ratio(st.lower(),str2.lower().split("(")[0].replace(".",""))

92

In [ ]:
st="liz carmouche"
str2="mohammed ragheb)"
